In [ ]:
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Flatten, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
tf.__version__
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pandas as pd

In [ ]:
!unzip "/content/drive/MyDrive/ColabNotebooks/Dataset/RMgx_data/Data.zip" -d "/content/" 

##**Prepare Train and Validation dataframes with label and file names in their respective columns**

In [ ]:

train = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Dataset/RMgx_data/train_labels.csv')   
train['imageId']= train['imageId'].astype(str)+'.jpg'

In [ ]:
validation = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Dataset/RMgx_data/validation_labels.csv')  
validation['imageId']= validation['imageId'].astype(str)+'.jpg'

##**Data Normalization**

In [ ]:
train_label_ids = []
dup_train_labels = []
for value in train['labelId'].values[:]:
  value=value.strip('][').split(', ')
  dup_train_labels+=value
  train_label_ids+=[value]

# List of all the Labels
train_labels_list=[]
for i in dup_train_labels:
  if i not in train_labels_list:
    train_labels_list+=[i]
train_labels_list+=["u'145'", "u'194'", "u'156'", "u'163'"]

# preparing data frame for a image with respective label values as 1 and rest of them as 0
train_df = pd.DataFrame()
train_df['labelId'] = train['labelId']
train_df['imageId'] = train['imageId']
for i in train_labels_list:
  values = []
  for j in train_label_ids:
    if i in j:
      values+=[1]
    else:
      values+=[0]
  train_df[i] = values
train_df.head()

,labelId,imageId,u'95',u'66',u'137',u'70',u'20',u'36',u'44',u'214',u'105',u'133',u'170',u'97',u'153',u'138',u'18',u'164',u'189',u'176',u'74',u'181',u'171',u'190',u'106',u'53',u'184',u'114',u'222',u'113',u'87',u'17',u'98',u'49',u'110',u'218',u'91',u'59',u'151',u'225',...,u'24',u'197',u'182',u'139',u'23',u'125',u'207',u'29',u'85',u'34',u'208',u'221',u'172',u'202',u'199',u'179',u'58',u'219',u'198',u'161',u'157',u'223',u'146',u'124',u'84',u'129',u'104',u'162',u'191',u'174',u'108',u'107',u'46',u'41',u'16',u'94',u'145',u'194',u'156',u'163'
0,"[u'95', u'66', u'137', u'70', u'20']",1.jpg,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"[u'36', u'66', u'44', u'214', u'105', u'133']",2.jpg,0,1,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"[u'170', u'66', u'97', u'153', u'105', u'138']",3.jpg,0,1,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"[u'18', u'66', u'44', u'153', u'164']",4.jpg,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"[u'189', u'66', u'176', u'137', u'153', u'74',...",5.jpg,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
validation_label_ids = []
dup_validation_labels = []
for value in validation['labels'].values[:]:
  value=value.strip('][').split(', ')
  dup_validation_labels+=value
  validation_label_ids+=[value]

# List of all the Labels  
validation_labels_list=[]
for i in dup_validation_labels:
  if i not in validation_labels_list:
    validation_labels_list.append(i)
validation_labels_list+=["u'46'", "u'96'", "u'161'", "u'162'", "u'16'"]

# preparing data frame for a image with respective label values as 1 and rest of them as 0
validation_df = pd.DataFrame()
validation_df['labels'] = validation['labels']
validation_df['imageId'] = validation['imageId']
for i in validation_labels_list:
  values = []
  for j in validation_label_ids:
    if i in j:
      values+=[1]
    else:
      values+=[0]
  validation_df[i] = values
validation_df.head()

,labels,imageId,u'62',u'17',u'66',u'214',u'105',u'137',u'85',u'95',u'164',u'20',u'204',u'184',u'122',u'19',u'186',u'180',u'44',u'154',u'190',u'222',u'153',u'226',u'53',u'171',u'111',u'70',u'14',u'98',u'12',u'175',u'54',u'138',u'116',u'176',u'56',u'210',u'61',u'106',...,u'199',u'27',u'107',u'140',u'227',u'157',u'215',u'39',u'139',u'198',u'23',u'127',u'174',u'34',u'93',u'1',u'152',u'221',u'29',u'104',u'58',u'206',u'146',u'173',u'86',u'202',u'3',u'149',u'185',u'223',u'68',u'156',u'129',u'83',u'195',u'46',u'96',u'161',u'162',u'16'
0,"[u'62', u'17', u'66', u'214', u'105', u'137', ...",1.jpg,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"[u'95', u'17', u'66', u'214', u'164', u'137', ...",2.jpg,0,1,1,1,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"[u'122', u'19', u'66', u'186', u'180', u'44', ...",3.jpg,0,0,1,0,0,0,0,0,0,1,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"[u'190', u'222', u'66', u'153', u'164', u'226'...",4.jpg,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"[u'62', u'66', u'153', u'171', u'111', u'137',...",5.jpg,1,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Storing column values for furthur use

In [ ]:
columns = train_df.columns.drop(['imageId','labelId'])
print(columns)

Index(['u'95'', 'u'66'', 'u'137'', 'u'70'', 'u'20'', 'u'36'', 'u'44'',
       'u'214'', 'u'105'', 'u'133'',
       ...
       'u'108'', 'u'107'', 'u'46'', 'u'41'', 'u'16'', 'u'94'', 'u'145'',
       'u'194'', 'u'156'', 'u'163''],
      dtype='object', length=228)


## Prepearing the train, test, valid dataset that model can understand

In [ ]:
data_generator = ImageDataGenerator(rescale=1./255,
                                        rotation_range=7,
                                        horizontal_flip=True,
                                       zoom_range=0.2)

train_dataset = data_generator.flow_from_dataframe(dataframe = train_df,
                                                        directory='/content/Data/train', x_col = "imageId", y_col = list(train_labels_list),
                                                        target_size = (128, 128),
                                                        batch_size = 32,
                                                        class_mode = 'raw', 
                                                       shuffle = True)


validate_dataset = data_generator.flow_from_dataframe(dataframe = validation_df,
                                                        directory='/content/Data/validation', x_col = "imageId", y_col = list(validation_labels_list),
                                                        target_size = (128, 128),
                                                        batch_size = 32,
                                                        class_mode = 'raw', 
                                                       shuffle = True)



test_generator = ImageDataGenerator(rescale = 1./255) 


Found 14720 validated image filenames.
Found 3000 validated image filenames.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 280 invalid image filename(s) in x_col="imageId". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
!mkdir test_images
#%cd test_images
!cp -r /content/Data/test /content/test_images/test

In [ ]:
test_dataset = test_generator.flow_from_directory("/content/test_images",
                                                  target_size=(128, 128),
                                                  batch_size=5, class_mode ='categorical',
                                                   shuffle = False)

Found 6895 images belonging to 1 classes.
